# L5b: Single Asset Geometric Brownian Motion Models
In this lecture, we introduce our first continuous-time model for asset prices, the Geometric Brownian Motion (GBM) model. This model is __widely__ used in finance due to its mathematical tractability and ability to capture key features of asset price dynamics.

> __Learning Objectives:__
>
> Fill me in later

Let's get started!
___

## Examples
Today, we will be using the following examples to illustrate key concepts:

___

## Single Asset Geometric Brownian Motion (GMB)
Geometric Brownian motion (GBM) is a stochastic differential equation describing the share price $S(t)$ 
as a random walk with drift and volatility proportional to $S(t)$:
$$
\begin{align*}
\frac{dS\left(t\right)}{S(t)} &= \mu\,{dt}+\sigma\,{dW(t)}\\
\end{align*}
$$
Here, $\mu\in\mathbb{R}$ denotes a constant drift (growth) parameter, $\sigma>0$ is a constant volatility parameter, and $dW$ represents a Wiener noise process. 

> __Wiener Noise__
> 
> Fill me in later.

The GBM model has the analytical solution:
$$
\boxed{
\begin{align*}
S(t) &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(t - t_{0}\right) + (\sigma\sqrt{t-t_{0}})\;{Z(0,1)}\Biggr]\\
\end{align*}}
$$
where $S_{\circ}$ is the initial share price at $t_{\circ}$, and $Z(0,1)$ is a standard normal random variable. The expectation and variance of a GBM model is given by:
$$
\begin{align*}
\mathbb{E}\left(S_{t}\right) &= S_{0}\;\exp\left(\mu\Delta{t}\right)\\
\text{Var}\left(S_{t}\right) &= S_{0}^{2}e^{2\mu\cdot\Delta{t}}\left[e^{\sigma^{2}{\Delta{t}}} - 1\right]
\end{align*}
$$

## Estimation of GBM Parameters
Fill me in later.

## GBM Trade Rule
Let's develop an expression that allows us to compute trade entry and exit rules based on the net present value (NPV) of a trade.

> __Scenario:__ Suppose we purchase $n_{o}$ shares of ticker `XYZ` at time $t=0$ (today) for $S_{0}$ USD/share.
> Then, sometime later, at $T=i\Delta{t}$, we sell all $n_{o}$ shares at a share price of $S_{T}$ USD/share, where $i\geq{0}$ is the number of time steps (integer) and $\Delta{t}$ is the time step (e.g., one trading day written in units of years).


This trade is a __long position__ because we profit if the share price goes up, i.e., $S_{T} > S_{0}$. Further, we can model it as an __abstract asset__ that has two cash flow events: the initial purchase at $t=0$ (entry) and the sale at $t=T$ (exit). The net present value (NPV) of this trade (written from our perspective) will have two cash flow events, the initial purchase at $t=0$ (entry) and the sale at $t=T$ (exit):
$$
\begin{align*}
\texttt{NPV}(\bar{r}, T) &= \underbrace{-n_{o}\;{S_{0}}}_{\text{entry (now)}} + \underbrace{n_{o}\;{S_{T}}\;\mathcal{D}_{T,0}^{-1}(\bar{r})}_{\text{exit (future)}}\quad\Longrightarrow\text{divide by initial investment}\;n_{o}\;{S_{0}}\\
\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}\;{S_{0}}} &= \frac{S_{T}\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - S_{0}}{S_{0}}\\
\underbrace{\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}\;{S_{0}}}}_{\text{fractional return}} &= \left(\frac{S_{T}}{S_{0}}\right)\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - 1\quad\blacksquare
\end{align*}
$$
where $\mathcal{D}^{-1}_{T,0}(\bar{r})$ is the (inverse) continuous discount factor for period $0\rightarrow{T}$ assuming a (constant) discount rate $\bar{r}$. 

Unlike last lecture, we used a binomial lattice models, let's now assume that the share price $S_{T}$ follows a GBM model. Then, we can substitute the GBM solution for $S_{T}$ into our NPV expression and simplify gives:
$$
\boxed{
\begin{align*}
\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}{S_{0}}} &= \exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)T + (\sigma\sqrt{T})\;{Z(0,1)}\Biggr]\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - 1\quad\blacksquare\\
\end{align*}}
$$

Let's take a closer look at this NPV expression, in the context of short versus long holding periods, our geometric Brownian motion model for the share price $S_{T}$, and postive and negative trade exists.

### Postive Trade Exit Probability
Unlike our binomial lattice model from last lecture, the NPV expression is not a __probability mass function__ (pmf) because the share price $S_{T}$ is a continuous random variable. However, we can still compute the probability that our trade meets some desired fractional return threshold $\texttt{NPV}(\bar{r}, T)/n_{o}S_{0} > r_{\star}$. Let's dig into this a bit more.

Let
$$
X \equiv \left(\mu-\frac{\sigma^{2}}{2}\right)T + \sigma\sqrt{T}\;Z,\qquad Z\sim \mathcal{N}(0,1),
$$
which implies
$$
\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}=e^{X}\mathcal{D}_{T,0}^{-1}(\bar r)-1.
$$
We want the scaled NPV to be (strictly) greater some specified fraction return $r_{\star}>0$, i.e., $\texttt{NPV}(\bar r,T)/n_o S_0> r_\star$. Thus,
$$
\begin{align*}
\mathbb{P}\!\left(\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}> r_\star\right)
& = \mathbb{P}\!\left(e^{X}\mathcal{D}_{T,0}^{-1}(\bar r)-1> r_\star\right)\\
& = \mathbb{P}\!\left(e^{X}> (1+ r_\star)\mathcal{D}_{T,0}(\bar r)\right)\\
& = \mathbb{P}\left(X>\ln\big((1+r_\star)\mathcal{D}_{T,0}(\bar r)\big)\right)
\end{align*}
$$
Ok, so here is the trick to this type of probability calculation. We need to write the probability expression in terms of the cumulative distribution function (CDF) of the standard normal random variable $Z$. To do this, we standardize the random variable $X$. Let $m=(\mu-\tfrac{1}{2}\sigma^2)T$ and $v=\sigma^2 T$. Then, solve $X$ for $Z$:
$$
\mathbb{P}\!\left(X>\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r)\right)
= \mathbb{P}\!\left(\underbrace{\frac{X-m}{\sigma\sqrt{T}}}_{Z\sim\mathcal{N}(0,1)}>\frac{\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r)-m}{\sigma\sqrt{T}}\right)
= 1-\Phi\!\left(\frac{\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r)-m}{\sigma\sqrt{T}}\right),
$$
or equivalently
$$
\boxed{\;
\mathbb{P}\!\left(\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}> r_\star\right)
= 1 - \Phi\!\left(\frac{\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r) - m}{\sigma\sqrt{T}}\right)\quad\blacksquare}
$$

where $\Phi(\cdot)$ is the cumulative distribution for a standard normal random variable. If we hold the $n_{o}$ shares for only a __short time__, e.g., a few days or even a few weeks, for realistic choices of the discount rate $\bar{r}$, the discount factor $\mathcal{D}_{T,0}(\bar{r})\approx{1}$, which simplifies the probability expression.

### Stop Loss Trade Exit Probability
Fill me in.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___